<a href="https://colab.research.google.com/github/ArshHp/DeepLearning_Fundamentals/blob/main/OpsB_Transformers_AskMe_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.4/640.4 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.6/228.6 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'

In [2]:
# Make sure you have a GPU running
!nvidia-smi

Thu Mar  9 08:51:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest main of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-kr01cfto/farm-haystack_439e426f63624ed1a5c5ed58d336dc3f
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-kr01cfto/farm-haystack_439e426f63624ed1a5c5ed58d336dc3f
  Resolved https://github.com/deepset-ai/haystack.git to commit e3503a92c9c889f9608e3b5d1bd587583ea15224
  Installing build dependencies ... done
  Getting requirements to

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [6]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

INFO:haystack.telemetry_2:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


Prepare the document: Add DocOps Document here

In [7]:
path_to_zip_file = 'drive/My Drive/MF/'
directory_to_extract_to = 'Opsb_Corpus'
doc_dir = path_to_zip_file+'/'+directory_to_extract_to



In [8]:
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=500,
    split_overlap=50,
    split_respect_sentence_boundary=True,
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
import os

indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
#indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["TextConverter"])

In [10]:
files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline.run_batch(file_paths=files_to_index)

INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.


Converting files:   0%|          | 0/500 [00:00<?, ?it/s]

Updating BM25 representation...:   0%|          | 0/500 [00:00<?, ? docs/s]

{'documents': [<Document: {'content': 'Get started\nContainerized Operations Bridge offers a monitoring solution that consolidates data from existing tools. It applies automated\ndiscovery, monitoring, analytics, and remediation to data across traditional, private, public, multi cloud, and container based\ninfrastructure. Using this solution, you can achieve the following goals:\nEvent consolidation - You can manage events from all your domains by consolidating them in Operations Bridge Manager\n(OBM). This consolidated view helps your IT operators to identify the causes of IT incidents and rectify issues. You can also\ncorrelate similar events, suppress unwanted events, or promote events based on your business requirements. \nNoise reduction - You can also get automatically correlated events in OBM. Driven by artificial intelligence (AI), automatic\ncorrelation reduces event related noise.\nCross domain business service driven out-of-the-box Reports. You can collect metrics, topology,

In [11]:
#search system will use a Retriever, so we need to initialize it. 
#A Retriever sifts through all the Documents and returns only the ones relevant to the question. 
#This tutorial uses the BM25 algorithm

from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)


In [12]:
?FARMReader

Object `FARMReader` not found.


In [13]:
#Initializing the Reader
#A Reader scans the texts it received from the Retriever and extracts the top answer candidates. 
data_dir="drive/My Drive/MF/Opsb_Corpus"
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
reader.distil_prediction_layer_from(reader, data_dir=data_dir, train_filename="answers.json", use_gpu=True)


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.data_handler.data_silo:
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|
 (o)(o)------'\ _ /     ( )
 
INFO:haystack.modeling.data_handler.data_silo:LOADING TRAIN DATA
INFO:haystack.modeling.data_handler.data_silo:==================
INFO:haystack.modeling.data_handler.data_silo:Loading train set from: drive/My Drive/MF/Opsb_Corpus/answers.json 


Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

Doing forward pass on teacher model:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.modeling.data_handler.data_silo:LOADING DEV DATA
INFO:haystack.modeling.data_handler.data_silo:=================
INFO:haystack.modeling.data_handler.data_silo:No dev set is being loaded
INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.modeling.data_handler.data_silo:LOADING TEST DATA
INFO:haystack.modeling.data_handler.data_silo:=================
INFO:haystack.modeling.data_handler.data_silo:No test set is being loaded
INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.modeling.data_handler.data_silo:DATASETS SUMMARY
INFO:haystack.modeling.data_handler.data_silo:================
INFO:haystack.modeling.data_handler.data_silo:Examples in train: 40
INFO:haystack.modeling.data_handler.data_silo:Examples in dev  : 0
INFO:haystack.modeling.data_handler.data_silo:Examples in test : 0
INFO:haystack.modeling.data_handler.data_silo:Total examples   : 40
INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

INFO:haystack.nodes.reader.farm:Saving reader model to ../../saved_models/Roberta


In [14]:
from haystack import Pipeline

querying_pipeline = Pipeline()
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])

The pipeline’s ready, you can now go ahead and ask a question!

In [15]:
prediction = querying_pipeline.run(
    query="Tell me about Opsb event correlation ?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)

Inferencing Samples:   0%|          | 0/4 [00:00<?, ? Batches/s]

In [16]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'Event Correlation offers the facility to automatically correlate events coming from OBM, by analyzing patterns', 'type': 'extractive', 'score': 0.09775859862565994, 'context': 'ata Lake.\nAutomatic Event Correlation offers the facility to automatically correlate events coming from OBM, by analyzing patterns. This\nautomatic eve', 'offsets_in_document': [{'start': 15525, 'end': 15635}], 'offsets_in_context': [{'start': 20, 'end': 130}], 'document_ids': ['5fc52982237960a004fc3919c6add461'], 'meta': {}}>,
             <Answer {'answer': 'Automatic Event Correlation\nAgentless Monitoring', 'type': 'extractive', 'score': 0.05896475166082382, 'context': 'boards (Business Value Dashboard)\nOPTIC Reporting \nAutomatic Event Correlation\nAgentless Monitoring*\nHyperscale Observability*\nContainerized OBM\nAnoma', 'offsets_in_document': [{'start': 166, 'end': 214}], 'offsets_in_context': [{'start': 51, 'end': 99}], 'document_ids': ['5fc52982237960a004fc3919c6add4

In [17]:
from haystack.utils import print_answers

print_answers(
    prediction,
    details="medium" ## Choose from `minimum`, `medium`, and `all`
)


Query: Tell me about Opsb event correlation ?
Answers:
[   {   'answer': 'Event Correlation offers the facility to automatically '
                  'correlate events coming from OBM, by analyzing patterns',
        'context': 'ata Lake.\n'
                   'Automatic Event Correlation offers the facility to '
                   'automatically correlate events coming from OBM, by '
                   'analyzing patterns. This\n'
                   'automatic eve',
        'score': 0.09775859862565994},
    {   'answer': 'Automatic Event Correlation\nAgentless Monitoring',
        'context': 'boards (Business Value Dashboard)\n'
                   'OPTIC Reporting \n'
                   'Automatic Event Correlation\n'
                   'Agentless Monitoring*\n'
                   'Hyperscale Observability*\n'
                   'Containerized OBM\n'
                   'Anoma',
        'score': 0.05896475166082382},
    {   'answer': 'itomdipulsar-proxy-575957cc44-tx422',
        'co

In [18]:
def opsb_chatbot(querying_pipeline,user_query,top_k):
  prediction = querying_pipeline.run(
    query=user_query,
    params={
        "Retriever": {"top_k": top_k*2},
        "Reader": {"top_k": top_k}
    }
  )
  return prediction

In [19]:
from haystack.utils import print_answers
result = opsb_chatbot(querying_pipeline,"Tell me about OPTIC Reporting capability please",5)
print_answers(
    result,
    details="all" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/3 [00:00<?, ? Batches/s]


Query: Tell me about OPTIC Reporting capability please
Answers:
[   <Answer {'answer': 'opticReporting', 'type': 'extractive', 'score': 0.1871112585067749, 'context': 'idge Suite offers. Users can enable one or multiple components.\n  # opticReporting: Reporting component contains OPTIC Data Lake, BVD and Collection S', 'offsets_in_document': [{'start': 798, 'end': 812}], 'offsets_in_context': [{'start': 68, 'end': 82}], 'document_ids': ['62442ee3901a92250ac1714b7f89954b'], 'meta': {}}>,
    <Answer {'answer': 'OPTIC Reporting uses OPTIC One to create reports and visualize the\nvisualize the real time data of the IT infrastructure', 'type': 'extractive', 'score': 0.15182016789913177, 'context': 'T environment. OPTIC Reporting uses OPTIC One to create reports and visualize the\nvisualize the real time data of the IT infrastructure. You can creat', 'offsets_in_document': [{'start': 1319, 'end': 1439}], 'offsets_in_context': [{'start': 15, 'end': 135}], 'document_ids': ['5fc52982237960a0

In [20]:
from haystack.utils import print_answers
result = opsb_chatbot(querying_pipeline,"what is BYOBI)",5)
print_answers(
    result,
    details="all", ## Choose from `minimum`, `medium`, and `all`
    max_text_len=None
)

Inferencing Samples:   0%|          | 0/2 [00:00<?, ? Batches/s]


Query: what is BYOBI)
Answers:
[   <Answer {'answer': 'Bring your own Business Intelligence', 'type': 'extractive', 'score': 0.5566558837890625, 'context': 't \nSynthetic Transaction \nReal User Monitor \nApplication\nBring your own Business Intelligence (BYOBI)\nYou can use third-party Business Intelligence (B', 'offsets_in_document': [{'start': 2449, 'end': 2485}], 'offsets_in_context': [{'start': 57, 'end': 93}], 'document_ids': ['bcd422f5476478db186938529574d177'], 'meta': {}}>,
    <Answer {'answer': 'Bring your own Business Intelligence', 'type': 'extractive', 'score': 0.27453532814979553, 'context': 't \nSynthetic Transaction \nReal User Monitor \nApplication\nBring your own Business Intelligence (BYOBI)\nYou can use third-party Business Intelligence (B', 'offsets_in_document': [{'start': 876, 'end': 912}], 'offsets_in_context': [{'start': 57, 'end': 93}], 'document_ids': ['bcd422f5476478db186938529574d177'], 'meta': {}}>,
    <Answer {'answer': 'business intelligence tool', 't

In [21]:
from haystack.utils import print_answers
result = opsb_chatbot(querying_pipeline,"what is Stakeholder Dashboard)",5)
print_answers(
    result,
    details="all", ## Choose from `minimum`, `medium`, and `all`
    max_text_len=None
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]


Query: what is Stakeholder Dashboard)
Answers:
[   <Answer {'answer': 'Business Value Dashboard', 'type': 'extractive', 'score': 0.4887005686759949, 'context': 'reate a deployment plan for installing\nStakeholder Dashboards (Business Value Dashboard) on a single node in K3s environment, you must consider the fo', 'offsets_in_document': [{'start': 540, 'end': 564}], 'offsets_in_context': [{'start': 63, 'end': 87}], 'document_ids': ['755f845a45a1e269f828b79b68cf8983'], 'meta': {}}>,
    <Answer {'answer': 'The stakeholder dashboard gives you visual information of live data using tables, charts, and\nwidgets', 'type': 'extractive', 'score': 0.3400340974330902, 'context': '\nStakeholder Dashboard\n: The stakeholder dashboard gives you visual information of live data using tables, charts, and\nwidgets. Real-time data can be ', 'offsets_in_document': [{'start': 677, 'end': 778}], 'offsets_in_context': [{'start': 25, 'end': 126}], 'document_ids': ['9e9ce6a04a1e87c3b832a726c62bd185'], 'meta':

In [22]:
result = opsb_chatbot(querying_pipeline,"Agentless Monitoring License requirement ?)",5)
print_answers(
    result,
    details="all", ## Choose from `minimum`, `medium`, and `all`
    max_text_len=None
)

Inferencing Samples:   0%|          | 0/3 [00:00<?, ? Batches/s]


Query: Agentless Monitoring License requirement ?)
Answers:
[   <Answer {'answer': 'Actions', 'type': 'extractive', 'score': 0.06156995892524719, 'context': 'd configuration item topology for infrastructure and applications.\n5\n. \nActions\n: Actions you can perform on a monitor: Edit, Copy, Alerts, and Delete', 'offsets_in_document': [{'start': 1814, 'end': 1821}], 'offsets_in_context': [{'start': 72, 'end': 79}], 'document_ids': ['2f5f43fb62e7c095563a3fc382d94601'], 'meta': {}}>,
    <Answer {'answer': "you don't need to apply for an additional license", 'type': 'extractive', 'score': 0.05108156055212021, 'context': "quest on OBM\n \nNote: \nTo use Agentless Monitoring, you don't need to apply for an additional license. The Sitescope license\nenables you to access Agen", 'offsets_in_document': [{'start': 3155, 'end': 3204}], 'offsets_in_context': [{'start': 51, 'end': 100}], 'document_ids': ['2dd94f007a54e3f6f0beedd2f0ed5002'], 'meta': {}}>,
    <Answer {'answer': 'Sitescope', 'typ

In [23]:
result = opsb_chatbot(querying_pipeline,"Explain OpsB",5)
print_answers(
    result,
    details="all", ## Choose from `minimum`, `medium`, and `all`
    max_text_len=None
)

Inferencing Samples:   0%|          | 0/4 [00:00<?, ? Batches/s]


Query: Explain OpsB
Answers:
[   <Answer {'answer': 'a monitoring solution that consolidates data from existing tools', 'type': 'extractive', 'score': 0.07524710893630981, 'context': 'ed\\nContainerized Operations Bridge offers a monitoring solution that consolidates data from existing tools. It applies automated\\ndiscovery, monitori', 'offsets_in_document': [{'start': 8453, 'end': 8517}], 'offsets_in_context': [{'start': 43, 'end': 107}], 'document_ids': ['db3c7a908596a86950425cb6cff51b70'], 'meta': {}}>,
    <Answer {'answer': 'genconfig', 'type': 'extractive', 'score': 0.05237694829702377, 'context': 'Configure the variables\nThe \ndbinit.sh\n \ngenconfig\n option allows you to configure the variables to create objects within Vertica while executing the\n', 'offsets_in_document': [{'start': 41, 'end': 50}], 'offsets_in_context': [{'start': 41, 'end': 50}], 'document_ids': ['c4cc82148352d0f119629caf5bff6b25'], 'meta': {}}>,
    <Answer {'answer': 'dbinit_conf.yaml\n file', 'type': 

In [24]:
result = opsb_chatbot(querying_pipeline,"What are the key capabilities of OpsB ?",5)
print_answers(
    result,
    details="all", ## Choose from `minimum`, `medium`, and `all`
    max_text_len=512
)

Inferencing Samples:   0%|          | 0/3 [00:00<?, ? Batches/s]


Query: What are the key capabilities of OpsB ?
Answers:
[   <Answer {'answer': 'automated\\ndiscovery, monitoring, analytics, and remediation', 'type': 'extractive', 'score': 0.42387521266937256, 'context': 'lidates data from existing tools. It applies automated\\ndiscovery, monitoring, analytics, and remediation to data across traditional, private, public,', 'offsets_in_document': [{'start': 8530, 'end': 8590}], 'offsets_in_context': [{'start': 45, 'end': 105}], 'document_ids': ['db3c7a908596a86950425cb6cff51b70'], 'meta': {}}>,
    <Answer {'answer': 'automated\\ndiscovery, monitoring, analytics, and remediation to data across traditional, private, public, multi cloud, and container based\\ninfrastructure', 'type': 'extractive', 'score': 0.3383371829986572, 'context': 'automated\\ndiscovery, monitoring, analytics, and remediation to data across traditional, private, public, multi cloud, and container based\\ninfrastructure', 'offsets_in_document': [{'start': 6777, 'end': 6931}], 'of